![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

### Mean Reversion
A number of financial signals can be thought to be mean-reverting (i.e. the spread between two assets prices). There are a number of ways to test for mean-reversion, but in this example we will assume it holds.

We use the research notebook to calculate the spread between an asset's price and its historical mean (past 30 days). Then, we calculate the standard deviation and determine which assets have moved more than one standard deviation below their average price. The theory is that they are due to revert back towards the mean, and so we can take advantage of this by taking long positions.

In [1]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
import numpy as np
qb = QuantBook()

symbols = {}
assets = ["SHY", "TLT", "SHV", "TLH", "EDV", "BIL",
                  "SPTL", "TBT", "TMF", "TMV", "TBF", "VGSH", "VGIT",
                  "VGLT", "SCHO", "SCHR", "SPTS", "GOVT"]

for i in range(len(assets)):
    symbols[assets[i]] = qb.AddEquity(assets[i],Resolution.Minute).Symbol

# Fetch history on our universe
df = qb.History(qb.Securities.Keys, 30, Resolution.Daily)
# Make all of them into a single time index.
df = df.close.unstack(level=0)
# Calculate the truth value of the most recent price being less than 1 std away from the mean
classifier = df.le(df.mean().subtract(df.std())).tail(1)
# Get indexes of the True values
classifier_indexes = np.where(classifier)[1]
# Get the Symbols for the True values
classifier = classifier.transpose().iloc[classifier_indexes].index.values
# Get the std values for the True values (used for Insight magnitude)
magnitude = df.std().transpose()[classifier_indexes].values
# Zip together to iterate over later
selected = zip(classifier, magnitude)

for x, y in selected:
    print(f'Symbol: {x}, Magnitude: {y}')

Symbol: BIL TT1EBZ21QWKL, Magnitude: 0.013359453149532978
Symbol: EDV TYCF240SL9PH, Magnitude: 3.3265208673173503
Symbol: GOVT V45XL2BVKU3P, Magnitude: 0.11821631840615553
Symbol: SHV TP8J6Z7L419H, Magnitude: 0.03615110356200481
Symbol: TLO TT1EBZ21QWKL, Magnitude: 0.6459442217433491
Symbol: TLT SGNKIKYGE9NP, Magnitude: 2.3672705126087825
Symbol: TMF UBTUG7D0B7TX, Magnitude: 1.7547680856690884
Symbol: VGLT UHVG8V7B7YAT, Magnitude: 1.3637322489563317
